# 7  Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst's time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

If you identify a type of data manipulation that isn’t anywhere in this book or elsewhere in the pandas library, feel free to share your use case on one of the Python mailing lists or on the pandas GitHub site. Indeed, much of the design and implementation of pandas have been driven by the needs of real-world applications.

In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

## 7.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. **One of the goals of pandas is to make working with missing data as painless as possible**. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is sufficient for most real-world use. For data with `float64` dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

We call this a *sentinel value*: when present, it indicates a missing (or *null*) value:



In [5]:
import pandas as pd
import numpy as np

In [6]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

The `isna` method gives us a Boolean Series with `True` where values are null:



In [7]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for *not available*. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA:

In [8]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [9]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [10]:
float_data = pd.Series([1, 2, None], dtype='float64')

float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [11]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

The pandas project has attempted to make working with missing data consistent across data types. Functions like `pandas.isna` abstract away many of the annoying details. See **[Table 7.1](https://wesmckinney.com/book/data-cleaning#tbl-table_na_method)** for a list of some functions related to missing data handling.



In [12]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is the same thing as doing:


In [13]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all NA, or only those rows or columns containing any NAs at all. `dropna` by default drops any row containing a missing value:



In [15]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all NA:



In [17]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Keep in mind that these functions return new objects by default and do not modify the contents of the original object.

To drop columns in the same way, pass `axis="columns"`:

In [18]:
data[4] = np.nan

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [19]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the `thresh` argument:



In [20]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))

df.iloc[:4, 1] = np.nan

df.iloc[:2, 2] = np.nan

df

,0,1,2
0,-1.265755,NaN,NaN
1,-1.033622,NaN,NaN
2,-2.334229,NaN,0.138440
3,-0.723374,NaN,-1.491639
4,-0.360054,-1.051234,0.728761
5,0.670821,0.371892,-1.487889
6,1.805276,-0.555939,-0.100938


In [21]:
df.dropna()

,0,1,2
4,-0.360054,-1.051234,0.728761
5,0.670821,0.371892,-1.487889
6,1.805276,-0.555939,-0.100938


In [23]:
df.dropna(thresh=2)

,0,1,2
2,-2.334229,NaN,0.138440
3,-0.723374,NaN,-1.491639
4,-0.360054,-1.051234,0.728761
5,0.670821,0.371892,-1.487889
6,1.805276,-0.555939,-0.100938


### Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:

In [24]:
df.fillna(0)

,0,1,2
0,-1.265755,0.000000,0.000000
1,-1.033622,0.000000,0.000000
2,-2.334229,0.000000,0.138440
3,-0.723374,0.000000,-1.491639
4,-0.360054,-1.051234,0.728761
5,0.670821,0.371892,-1.487889
6,1.805276,-0.555939,-0.100938


Calling fillna with a dictionary, you can use a different fill value for each column:



In [25]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.265755,0.500000,0.000000
1,-1.033622,0.500000,0.000000
2,-2.334229,0.500000,0.138440
3,-0.723374,0.500000,-1.491639
4,-0.360054,-1.051234,0.728761
5,0.670821,0.371892,-1.487889
6,1.805276,-0.555939,-0.100938


The same interpolation methods available for reindexing (see Table 5.3) can be used with `fillna`:

In [26]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = np.nan

df.iloc[4:, 2] = np.nan

df

,0,1,2
0,-1.197100,0.116138,0.672365
1,-0.248726,1.023342,-0.229456
2,-0.961917,NaN,-2.154188
3,-0.886859,NaN,0.155661
4,0.800382,NaN,NaN
5,0.341091,NaN,NaN


In [27]:
df.fillna(method="ffill")

/var/folders/fn/3w5085ns7nv4v62cfvj7m9sm0000gp/T/ipykernel_81114/3944122520.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")


,0,1,2
0,-1.197100,0.116138,0.672365
1,-0.248726,1.023342,-0.229456
2,-0.961917,1.023342,-2.154188
3,-0.886859,1.023342,0.155661
4,0.800382,1.023342,0.155661
5,0.341091,1.023342,0.155661


In [28]:
df.fillna(method="ffill", limit=2)

/var/folders/fn/3w5085ns7nv4v62cfvj7m9sm0000gp/T/ipykernel_81114/1627181726.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", limit=2)


,0,1,2
0,-1.197100,0.116138,0.672365
1,-0.248726,1.023342,-0.229456
2,-0.961917,1.023342,-2.154188
3,-0.886859,1.023342,0.155661
4,0.800382,NaN,0.155661
5,0.341091,NaN,0.155661


With `fillna` you can do lots of other things such as simple data imputation using the median or mean statistics:

In [29]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

See **[Table 7.2](https://wesmckinney.com/book/data-cleaning#tbl-table_fillna_function)** for a reference on fillna function arguments.


## 7.2 Data Transformation

So far in this chapter we’ve been concerned with handling missing data. Filtering, cleaning, and other transformations are another class of important operations.

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:


In [30]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a Boolean Series indicating whether each row is a duplicate (its column values are exactly equal to those in an earlier row) or not:



In [31]:

data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame with rows where the `duplicated` array is `False` filtered out:

In [32]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the `"k1"` column:

In [33]:
data["v1"] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [35]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:


In [38]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:



In [40]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0
